In [ ]:
from google.colab import files
uploaded = files.upload()
file_name = "AntiScam_annotated.txt"
data = uploaded[file_name].decoxqde("latin-1").split("\r\n")

Saving AntiScam_annotated.txt to AntiScam_annotated (1).txt


In [ ]:
data[0:10]

['0\tHello my name is NAMESLOTA and I am with Amazon customer support.\tgreeting\tname\t\t\t\t\t\t\t\t\t\t',
 '1\tHello NAMESLOTA.\t\t\tgreeting\tname\t\t\t\t\t\t\t\t',
 '1\tI recently purchased a heater but it has not arrived\t\t\tnonresponsive_statement\torder_detail\t\t\t\t\t\t\t\t',
 "0\tI'm sorry to hear that.\tapology\torder_detail\t\t\t\t\t\t\t\t\t\t",
 '0\tCan I have your name and address to look this up?\telicitation\taddress\t\t\t\t\t\t\t\t\t\t',
 '1\tWhy is my address necessary NAMESLOTA?\t\t\topen_question\taddress\t\t\t\t\t\t\t\t',
 '0\t"Due to our volume, we have certain guidelines on information needed for order inquiries."\tresponsive_statement\taddress\t\t\t\t\t\t\t\t\t\t',
 '1\tAlright before I proceed may I have your full name for personal record.\t\t\telicitation\tname\t\t\t\t\t\t\t\t',
 "0\tI'm sorry that is against our policies.\trefusal\tname\t\t\t\t\t\t\t\t\t\t",
 '0\tI can give you my employee number if you need.\tnonresponsive_statement\tidentity\t\t\t\t\t\t\t

In [ ]:
lis=data[10].strip().split("\t")

In [ ]:
for l in lis:
  if l=='': continue
  print(normalizeString(l.strip()))


ok what is your employee number ?
open question
identity


In [ ]:
from google.colab import files
uploaded = files.upload()
file_name = "AntiScam_all.txt"
old_data = uploaded[file_name].decode("latin-1").split("\r\n")

Saving AntiScam_all.txt to AntiScam_all.txt


In [ ]:
old_data[0]

'0\tHello my name is NAMESLOT and I am with Amazon customer support.'

In [ ]:
import os
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
    
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s
    
def extractSentencePairs(data):
  qa_pairs = []
  last_bot_line=""
  last_client_line=""
  for i in range(len(data)-1):
    inputLine = data[i].split("\t")
    targetLine = data[i+1].split("\t")
    #Checking if not blank case
    if (inputLine[0]=='1' or inputLine[0]=='0') and (targetLine[0]=='1' or targetLine[0]=='0'):
      if inputLine[0]=='0' and targetLine[0]=='1':
        qa_pairs.append([inputLine[1], targetLine[1]])
        last_bot_line=inputLine[1]
      elif inputLine[0]=='1' and targetLine[0]=='0':
        qa_pairs.append([inputLine[1], targetLine[1]])
        last_client_line=inputLine[1]
      elif inputLine[0]=='1' and targetLine[0]=='1':
        qa_pairs.append([last_bot_line, targetLine[1]])
      elif inputLine[0]=='0' and targetLine[0]=='0':
        qa_pairs.append([last_client_line, targetLine[1]])
  return qa_pairs
  

In [ ]:
# map--> mul intent keys: list of text
         # slots: set of diff slots


In [ ]:
ip=data[33].strip()
print(ip)
print(type(ip))
ip==''

1	Yes
<class 'str'>


False

In [ ]:
inputLine = data[34].strip().split("\t")


<class 'list'>


False

In [ ]:
def join_data(data):
  lis=[]
  for d in data:
    if d=='': continue
    else: lis.append(d)
  return lis

In [ ]:
bot_text={
    "intents":{},
    "slots":{}
} #bot text - 0

user_text={
    "intents":{},
    "slots":{}
} #user text - 1


In [ ]:
spl_case=[]
def map_data(data):
  for i in range(len(data)):
  # for i in range(20):
    inputLine = data[i].strip().split("\t")
    inputLine= join_data(inputLine)
    if(len(inputLine)<4):
      spl_case.append(data[i])
      continue
    text=normalizeString(inputLine[1])
    intent= inputLine[2]
    slot= inputLine[3]
    if(inputLine[0]=='0'):
      if intent not in bot_text["intents"].keys():
        bot_text["intents"][intent]=[]
      bot_text["intents"][intent].append(text)

      if slot not in bot_text["slots"].keys():
        bot_text["slots"][slot]=[]
      bot_text["slots"][slot].append(text)
    elif(inputLine[0]=='1'):
      if intent not in user_text["intents"].keys():
        user_text["intents"][intent]=[]
      user_text["intents"][intent].append(text)

      if slot not in user_text["slots"].keys():
        user_text["slots"][slot]=[]
      user_text["slots"][slot].append(text)

map_data(data)

In [ ]:
bot_text['intents'].keys()

dict_keys(['greeting', 'apology', 'elicitation', 'responsive_statement', 'refusal', 'nonresponsive_statement', 'yes_no_question', 'provide', 'hold', 'negative_answer', 'positive_answer', 'open_question', 'thanking', 'response_to_thanking', 'closing'])

In [ ]:
type(bot_text['intents']['greeting'])
# bot_text['intents']['greeting']

list

In [ ]:
bot_text['slots'].keys()

dict_keys(['name', 'order_detail', 'address', 'identity', 'others', 'payment', 'card_num', 'order_update', 'phone_num', 'card_info', 'card_csv', 'account_detail', 'card_date'])

In [ ]:

import json
  
with open('user_text.txt', 'w') as convert_file:
     convert_file.write(json.dumps(user_text))

In [ ]:
bot_text['intents'].keys()

AttributeError: ignored